<span style="color:#888888">Copyright (c) 2014-2025 National Technology and Engineering Solutions of Sandia, LLC. Under the terms of Contract DE-NA0003525 with National Technology and Engineering Solutions of Sandia, LLC, the U.S. Government retains certain rights in this software.     Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:</span>

<span style="color:#888888">1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.</span>

<span style="color:#888888">2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.</span>

<span style="color:#888888">THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.</span>

# <span style="color:#0054a8">**Demo 1:**</span> <span style="color:#555555">Anomaly Detection using Passersby</span>

This notebook shows the use of one of Tracktable's built-in applications: simple anomaly detection.  

Here, we define an anomaly as a trajectory that does not have any other trajectories nearby.  We sample the trajectories at 4 evenly spaced points each (the `num_control_points` argument to `anomaly_detection()`) and search for passers-by.

In [ ]:
from tracktable.render.render_trajectories import render_trajectories
from tracktable.applications.anomaly_detection import anomaly_detection
import tracktable.examples.tutorials.tutorial_helper as tutorial

## Import Historical and Test Datasets

For this demo, we will use one week of maritime traffic in NY Harbor in December 2020$^1$ as our historical dataset, and the following day of maritime traffic in NY Harbor $^2$ for our test trajectory dataset.

In [ ]:
historical_trajectories = tutorial.get_trajectory_list('anomaly-historical')
test_trajectories = tutorial.get_trajectory_list('anomaly-test')

## Detect Anomalies

We send the anomaly detection function our historical and test trajectories, and for each test trajectory it detects how many historical trajectories passed by all `num_control_points` points equally-spaced along the trajectory (defaulted to 4), within a radius of `nearness_radius` kilometers (defaulted to 5km) of each point.

In [ ]:
%%time

nearby_trajectory_indices = anomaly_detection(test_trajectories,
                                                historical_trajectories=historical_trajectories,
                                                nearness_radius=1,
                                                num_control_points=4,
                                                filename="test.log")

### Store our anomaly score as a trajectory property.

We can create new properties to be stored with a given trajectory.  Let's store the number of passersby from the historical trajectories as an "anomaly score", where 0 is the most anomalous, as this means no historical trajectories passed along the same route.

In [ ]:
for i, traj in enumerate(test_trajectories):
    traj.set_property('anomalous_score', len(nearby_trajectory_indices[i]))

### Sort the trajectories by their anomaly score.

In [ ]:
sorted_trajectories = sorted(enumerate(test_trajectories), key=lambda x: x[1].properties['anomalous_score'])

## Visualize Anomalies

### Visualize all anomalous trajectories

You can scroll and zoom to examine each anomalous trajectory more closely.

In [ ]:
render_trajectories([trajectory for trajectory in test_trajectories if trajectory.properties['anomalous_score'] == 0])

### Visualize the `i`-th most anomalous trajectory, where `i = 0` is the most and `i = -1` is the least.

In [ ]:
# Change i to change the trajectory being visualized in the following cells.
i = -1

#### Render the trajectory alone.

In [ ]:
canvas = render_trajectories(sorted_trajectories[i][1], show=True)

#### Render again with historical passersby.

Create a list of the historical trajectories that passed by the same route as this trajectory.

In [ ]:
passersby = []
for index in nearby_trajectory_indices[sorted_trajectories[i][0]]:
    passersby.append(historical_trajectories[index])

How many historical passersby does this trajectory have?

In [ ]:
len(passersby)

Let's render the first twenty alongside the `i`-th most anomalous test trajectory.

In [ ]:
render_trajectories(passersby[:20], canvas=canvas)

<span style="color:gray">$^1$ Bureau of Ocean Energy Management (BOEM) and National Oceanic and Atmospheric Administration (NOAA). MarineCadastre.gov. <i>AIS Data for 2020.</i> Retrieved May 2021 from [marinecadastre.gov/data](https://marinecadastre.gov/data/).  US coastal maritime traffic trimmed down to June 30, 2020.</span>
   <br>
   <br>
<span style="color:gray">$^2$ Bureau of Ocean Energy Management (BOEM) and National Oceanic and Atmospheric Administration (NOAA). MarineCadastre.gov. <i>AIS Data for 2020.</i> Retrieved May 2021 from [marinecadastre.gov/data](https://marinecadastre.gov/data/).  US coastal maritime traffic trimmed down to before noon on July 1, 2020.</span>
